#Deep Learning con Python
`Autor: Erwing FC 
~erwingforerocastro@gmail.com`



In [ ]:
# pip install vaex #comando para instalar la libreria (recuerde eliminar el comentario)

In [ ]:
# pip install ipython --upgrade #comando para actualizar ipython si es necesario (recuerde eliminar el comentario)

In [ ]:
import vaex
import numpy as np
import matplotlib.pylab as plt

##Dataframes
similar a pandas se representa por `df`, sus caracteristicas son:
1. al visualizar los datos podemos ver que es muy similar al formato excel
2. respaldado por matrices numpy
3. expreciones como `df.x`, `df['x']` o `df.col.x`
4. las columnas o expreciones no realizan nada hasta necesitar el calculo
5. Un conjunto de selecciones que se pueden usar para explorar el conjunto de datos, por ejemplo `df.select(df.x < 0)`
6. Filtrar los datos de un dataframe que no copia los datos, `df_negative = df[df.x < 0]`

In [ ]:
df=vaex.example
df
# x, y, z (posiciones), vx, vy, vz (velocidades), E (energía), L (momento angular) y un id (subgrupo de muestras) 

In [ ]:
#columnas
df.x
#valores
df.x.values

In [ ]:
#columna virtual solo es calculada cuando se requiere
df['r']=np.sqrt(df.x**2 + df.y**2 + df.z**2)
df[['x','y','z','r']]

In [ ]:
#selecciones y filtrado
df.select(df.x<0) # solo se usa cuando se necesita sobre la columna x
df.evaluate(df.x,selection=True)

In [ ]:
#tambien se pueden crear filtered dataframes sin copiar los datos
df_negative = df[df.x < 0]
df_negative[['x', 'y', 'z', 'r']]

In [ ]:
#estadisticas sobre cuadriculas
df.count(),df.mean(df.x),df.mean(df.x,selection=True) #cantidad,media de la columna x, media de la columna x menores a 0

In [ ]:
counts_x=df.count(binby=df.x,limits=[-10,10],shape=64) #binby es similar a group by de sql
                                                       #esto trae el conteo o cantidad de 64 elementos, con valores de -10 a 10
plt.plot(np.linspace(-10,10,64),counts_x)
plt.show()

In [ ]:
xycounts = df.count(binby=[df.x, df.y], limits=[[-10, 10], [-10, 20]], shape=(64, 128))
plt.imshow(xycounts.T, origin='lower', extent=[-10, 10, -10, 20])
plt.show()

In [ ]:
v=np.sqrt(df.vx**2+df.vy**2+df.vz**2)
xy_mean_v=df.mean(v,binby=[df.x,df.y],limits=[[-10,10],[-10,20]],shape=(64,128))
plt.imshow(xy_mean_v.T, origin='lower', extent=[-10, 10, -10, 20])
plt.show()

In [ ]:
#tambien se puede construir un dataframe apartir de variables numpy
x=np.arange(5)
y=x**2
df=vaex.from_arrays(x=x,y=y)
df

In [ ]:
#descargar datos de s3 
nyctaxi = vaex.open('s3://vaex/taxi/yellow_taxi_2009_2015_f32.hdf5?anon=true')
nyctaxi.head(5)

In [ ]:
#plotting
df = vaex.example()
df.plot1d(df.x, limits='99.7%')

In [ ]:
#grafica la media E de cada x
df.plot1d(df.x, what='mean(E)', limits='99.7%') # es igual a vaex.stat.mean(df.E)

In [ ]:
np.log(vaex.stat.mean(df.x)/vaex.stat.std(df.x)) # objeto estadistico
df.plot1d(df.x, what=np.clip(np.log(-vaex.stat.mean(df.E)), 11, 11.4), limits='99.7%') # se puede pasar al argumento what

In [ ]:
#otra forma de realizarlo
limits = [-30, 30]
shape  = 64
meanE  = df.mean(df.E, binby=df.x, limits=limits, shape=shape)
grid   = np.clip(np.log(-meanE), 11, 11.4)
df.plot1d(df.x, grid=grid, limits=limits, ylabel='clipped E')

In [ ]:
#trazado en 2D
df.plot(df.x, df.y, what=vaex.stat.mean(df.E)**2, limits='99.7%')